In [1]:
!pip install -U sentence_transformers
!pip install xformers

In [2]:
import os
import json
from sentence_transformers import SentenceTransformer
#from langchain_community.document_loaders import PyPDFLoader
#from langchain_text_splitters import RecursiveCharacterTextSplitter

## Sentence transfomer baseline model
We choose `stella_en_400M_v5` model from [MTEB leaderboard](https://huggingface.co/spaces/mteb/leaderboard).

In [28]:
query_prompt_name = "s2p_query"
queries = [
    "What material is the rear wing of the 718 Cayman GT4 RS made of?",
    "What is the unique feature of the Cayenne Turbo GT?",
    "What customization options are available for the Taycan Cross Turismo?",
    "What is the combined CO₂ emissions for the 718 Cayman GTS 4.0?",
    "What is the GTS model?",
    "What is the history of Cayenne?"
]

docs = [
    "Carbon fiber reinforced plastic (CFRP).",
    "It offers 471 kW (640 PS) and is optimized for high performance.",
    "Exclusive paint finishes, interior trims, and wheel designs.",
    "The 718 Cayman GTS 4.0 produces zero CO₂ emissions because it is fully electric.",
    "GTS model is a great coaching system for MCQ practicing, which G stands for Guardian, T for Teacher and S for Student.",
    "The history of Cayenne can be first traced back to pre-Columbian times."
]

# ！The default dimension is 1024, if you need other dimensions, please clone the model and modify `modules.json` to replace `2_Dense_1024` with another dimension, e.g. `2_Dense_256` or `2_Dense_8192` !
# on gpu
model = SentenceTransformer("dunzhang/stella_en_400M_v5", trust_remote_code=True).cuda()
# you can also use this model without the features of `use_memory_efficient_attention` and `unpad_inputs`. It can be worked in CPU.
# model = SentenceTransformer(
#     "dunzhang/stella_en_400M_v5",
#     trust_remote_code=True,
#     device="cpu",
#     config_kwargs={"use_memory_efficient_attention": False, "unpad_inputs": False}
# )
query_embeddings = model.encode(queries, prompt_name=query_prompt_name)
doc_embeddings = model.encode(docs)
print(query_embeddings.shape, doc_embeddings.shape)

Some weights of the model checkpoint at dunzhang/stella_en_400M_v5 were not used when initializing NewModel: ['new.pooler.dense.bias', 'new.pooler.dense.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(6, 1024) (6, 1024)


### Issue 1: positive example sentence pairs but low similariy

In [29]:
for i in range(3):
    similarities = model.similarity(query_embeddings[i], doc_embeddings[i])
    print(f"Uesr Query: {queries[i]}")
    print(f"Answer: {docs[i]}")
    print(f"Similarity score: {similarities.data.cpu().numpy()[0][0]} \n\n")

Uesr Query: What material is the rear wing of the 718 Cayman GT4 RS made of?
Answer: Carbon fiber reinforced plastic (CFRP).
Similarity score: 0.5579701066017151 


Uesr Query: What is the unique feature of the Cayenne Turbo GT?
Answer: It offers 471 kW (640 PS) and is optimized for high performance.
Similarity score: 0.4106892943382263 


Uesr Query: What customization options are available for the Taycan Cross Turismo?
Answer: Exclusive paint finishes, interior trims, and wheel designs.
Similarity score: 0.5345098376274109 




### Issue 2: negative example sentence pairs but high similariy

In [27]:
for i in range(3, 6):
    similarities = model.similarity(query_embeddings[i], doc_embeddings[i])
    print(f"Uesr Query: {queries[i]}")
    print(f"Answer: {docs[i]}")
    print(f"Similarity score: {similarities.data.cpu().numpy()[0][0]} \n\n")

Uesr Query: What is the combined CO₂ emissions for the 718 Cayman GTS 4.0?
Answer: The 718 Cayman GTS 4.0 produces zero CO₂ emissions because it is fully electric.
Similarity score: 0.8328068256378174 


Uesr Query: What is the GTS model?
Answer: GTS model is a great coaching system for MCQ practicing, which G stands for Guardian, T for Teacher and S for Student.
Similarity score: 0.7170215845108032 


Uesr Query: What is the history of Cayenne?
Answer: The history of Cayenne can be first traced back to pre-Columbian times.
Similarity score: 0.8268212676048279 




## The results suggest that we can further fine-tune the pretrained models.